In [2]:
from smach_utils2 import *
img=rgbd.get_image()

In [ ]:
img=rgbd.get_image()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s")  # or yolov5n - yolov5x6, custom

In [ ]:
import torch
import torch.backends.cudnn as cudnn

from yolov5.models.experimental import attempt_load
from yolov5.utils.dataloaders import LoadStreams, LoadImages
from yolov5.utils.general import *
#from yolov5.utils.general import check_img_size, check_requirements, check_imshow, colorstr, non_max_suppression, \
    #apply_classifier, scale_boxes, scale_segments, xyxy2xywh, strip_optimizer, set_logging, increment_path#, save_one_box
from yolov5.utils.augmentations import letterbox
from yolov5.utils.plots import colors, Annotator
from yolov5.utils.torch_utils import * #select_device, load_classifier, time_sync


In [ ]:
device = select_device('')

In [3]:
import torch
import torch.backends.cudnn as cudnn
from yolov5.models.experimental import attempt_load
from yolov5.utils.dataloaders import LoadStreams, LoadImages
from yolov5.utils.general import *



WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [4]:

from yolov5.models.experimental import attempt_load
from yolov5.utils.dataloaders import LoadStreams, LoadImages
from yolov5.utils.general import *

In [5]:
from yolov5.utils.augmentations import letterbox
from yolov5.utils.plots import colors, Annotator
from yolov5.utils.torch_utils import * #select_device, load_c

In [8]:
device = select_device('')
model=attempt_load('/home/roboworks/catkin_extras/src/yolov5_ros/scripts/yolov5/ycb.pt',device)

YOLOv5 🚀 2023-7-23 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1080 with Max-Q Design, 8109MiB)

Fusing layers... 
Model summary: 369 layers, 21138024 parameters, 0 gradients


In [9]:
model.names

{0: '001_chips_can',
 1: '002_master_chef_can',
 2: '003_cracker_box',
 3: '004_sugar_box',
 4: '005_tomato_soup_can',
 5: '006_mustard_bottle',
 6: '007_tuna_fish_can',
 7: '008_pudding_box',
 8: '009_gelatin_box',
 9: '010_potted_meat_can',
 10: '011_banana',
 11: '012_strawberry',
 12: '013_apple',
 13: '014_lemon',
 14: '015_peach',
 15: '016_pear',
 16: '017_orange',
 17: '018_plum',
 18: '019-a_pitcher_base',
 19: '019-b_pitcher_base_v2',
 20: '020_pitcher_lid',
 21: '021_bleach_cleanser',
 22: '022_windex_bottle',
 23: '024_bowl',
 24: '025_mug',
 25: '026_sponge',
 26: '029_plate',
 27: '030_fork',
 28: '031_spoon',
 29: '033_spatula',
 30: '040_large_marker',
 31: '050_medium_clamp',
 32: '051_large_clamp',
 33: '052_extra_large_clamp',
 34: '053-a_mini_soccer_ball',
 35: '053-b_mini_soccer_ball_v2',
 36: '054_softball',
 37: '055_baseball',
 38: '056_tennis_ball',
 39: '057_racquetball',
 40: '058_golf_ball',
 41: '059_chain',
 42: '061_foam_brick',
 43: '062_dice',
 44: '063

In [10]:
img = torch.from_numpy(img).to(device)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load model and weights to GPU via cuda if available
model=attempt_load('/home/roboworks/catkin_extras/src/yolov5_ros/scripts/yolov5/ycb.pt',device)

Fusing layers... 
Model summary: 369 layers, 21138024 parameters, 0 gradients


In [12]:
model.info()

Model summary: 369 layers, 21138024 parameters, 0 gradients, 49.1 GFLOPs


In [13]:
img = torch.from_numpy(rgbd.get_image()).to(device)
img = img / 255.0

In [14]:
iimmgg=img.unsqueeze(0)
iimmgg.shape


torch.Size([1, 480, 640, 3])

In [15]:
im=torch.moveaxis(iimmgg,3,1)

In [16]:
im.shape

torch.Size([1, 3, 480, 640])

In [18]:
res= model(im)

In [19]:
res[0].shape

torch.Size([1, 18900, 72])

In [20]:
augment= False
pred = model(im, augment=augment, visualize=False)[0]
conf_thres=0.5  # confidence threshold
iou_thres=0.45  # NMS IOU threshold
max_det=1000
classes=0
agnostic_nms=False
D = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms,max_det)

In [21]:
D

[tensor([], device='cuda:0', size=(0, 6))]

In [ ]:
len(det)

In [ ]:
modelc = load_classifier(name='resnet101', n=2) 

In [ ]:
import yolov5

In [ ]:
yolov5.utils.

In [ ]:
read_yaml('/known_locations.yaml')
#read_yaml('/known_locations_sim.yaml')

In [ ]:
df=yaml_to_df('/known_locations.yaml')
#df=yaml_to_df('/known_locations_sim.yaml')

In [ ]:
xys=[]
xys.append(df[df['child_id_frame']=='bedroom'][['x','y']].values.ravel())
xys.append(df[df['child_id_frame']=='living_room'][['x','y']].values.ravel())
xys.append(df[df['child_id_frame']=='dining_room'][['x','y']].values.ravel())
xys.append(df[df['child_id_frame']=='kitchen'][['x','y']].values.ravel())
room_names=['bedroom','living_room','dining_room','kitchen']
xys, room_names

In [ ]:
humanpose=detect_human_to_tf()  #make sure service is running
humanpose

In [ ]:
tf_man.getTF('human')

In [ ]:
import matplotlib.pyplot as plt

map_msg= rospy.wait_for_message('/prohibition_layer_map', OccupancyGrid)
#map_msg= rospy.wait_for_message('/augmented_map', OccupancyGrid)
inflated_map= np.asarray(map_msg.data)
#cv2.imwrite('/home/roboworks/Pictures/inflatedmap.png',inflated_map)

In [ ]:

img_map=inflated_map.reshape((map_msg.info.width,map_msg.info.height))


plt.imshow(img_map)

In [ ]:
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]

In [ ]:
res=segmentation_server.call()
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]

if len(res.poses.data)==0: print( 'failed')
else:
    print('object found')
    
    poses=np.asarray(res.poses.data)
    poses=poses.reshape((int(len(poses)/3) ,3     )      )  
    num_objs=len(poses)
    print (num_objs)
    for i,pose in enumerate(poses):
        #print (f'Occupancy map at point object {i}-> pixels ',origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m), img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)])
        point_name=f'object_{i}'
        tf_man.pub_static_tf(pos=pose, point_name=point_name, ref='head_rgbd_sensor_rgb_frame')## which object to choose   #TODO
        rospy.sleep(0.3)
        tf_man.change_ref_frame_tf(point_name=point_name, new_frame='map')
        rospy.sleep(0.3)
        pose,_= tf_man.getTF(point_name)
        print (f'Occupancy map at point object {i}-> pixels ',origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m), img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)])
        if img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]!=0:#### Yes axes seem to be "flipped" !=0:
            print ('reject point, most likely part of arena, occupied inflated map')
            tf_man.pub_static_tf(pos=[0,0,0], point_name=point_name, ref='head_rgbd_sensor_rgb_frame')
            num_objs-=1
        print (f"object found at robot coords.{pose} ")

print(num_objs)

In [ ]:
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]
humanpose=detect_human_to_tf()  #make sure service is running

In [ ]:
if humanpose== False:
    
    print ('no human ')
    
else : 
    human_pose,_=tf_man.getTF('human')

    pose=human_pose[:2]
    
    if img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]!=0:#### Yes axes seem to be "flipped" !=0:
            print ('reject point, most likely part of the audience, outside of the arena map')
            
    dists=(pose-np.asarray(xys))
    human_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'human in {human_room}')
    
    robot_pose=get_robot_px()
    dists=(robot_pose-np.asarray(xys))
    robot_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'Robot  in {robot_room}')
    
    if robot_room != human_room: print('maybe false positive... ignoring... ')
    

In [ ]:
def draw_at (px,py, contoured):
    contoured[px-5:px+5,py-5:py+5]=200
    return contoured
    

In [ ]:
iimg=draw_at(origin_map_img[1]+ round(pose[1]/pix_per_m), origin_map_img[0]+ round(pose[0]/pix_per_m) , img_map )
#iimg=draw_at(origin_map_img[1], origin_map_img[0] , img_map )

In [ ]:
plt.imshow(iimg)

In [ ]:
pose=human_pose
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]

print ('Occupancy map at point human-> pixels ',origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m), img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)])




In [ ]:
xys

In [ ]:
human_pose,_=tf_man.getTF('human')
human_pos=human_pose[:2]

In [ ]:
human_pos=human_pose[:2]

In [ ]:
#human_pos=np.asarray([10.6,-2.33])

In [ ]:
dists=(human_pos-np.asarray(xys))
room_names[np.linalg.norm(dists, axis=1).argmin()]

In [ ]:

xys.append(df[df['child_id_frame']=='bedroom'][['x','y']].values)

In [ ]:

x=df[df['child_id_frame']=='bedroom'][['x','y']].values

In [ ]:
x.ravel().shape

In [ ]:
img_map[300,285:315]

In [ ]:
import cv2
cv2.imwrite('/home/roboworks/Pictures/imgmap.png', img_map)
img_map_rooms=cv2.imread('/home/roboworks/Pictures/imgmap.png')

In [ ]:
import cv2
img_map_rooms=cv2.imread('/home/roboworks/Pictures/imgmap_rooms.png')

In [ ]:
plt.imshow(img_map_rooms)

In [ ]:

iimg=draw_at(origin_map_img[1]+ round(pose[1]/pix_per_m), origin_map_img[0]+ round(pose[0]/pix_per_m) , img_map_rooms )

In [ ]:
plt.imshow(iimg)

In [ ]:
img_map_rooms=cv2.imread('/home/roboworks/catkin_extras/src/tmc_wrs_gazebo_world/maps/rc2023/map.pgm')

img_map_rooms[origin_map_img[1]+ round(pose[1]/pix_per_m), origin_map_img[0]+ round(pose[0]/pix_per_m)]

In [ ]:
#living_room

In [ ]:
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]
humanpose=detect_human_to_tf()  #make sure service is running
if humanpose== False:
    
    print ('no human ')
    
else : 
    human_pose,_=tf_man.getTF('human')

    pose=human_pose[:2]
    
    if img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]!=0:#### Yes axes seem to be "flipped" !=0:
            print ('reject point, most likely part of the audience, outside of the arena map')
            
    dists=(pose-np.asarray(xys))
    human_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'human in {human_room}')
    
    robot_pose=get_robot_px()
    dists=(robot_pose-np.asarray(xys))
    robot_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'Robot  in {robot_room}')
    
    if robot_room != human_room: print('maybe false positive... ignoring... ')
    

In [ ]:
img_map_rooms=cv2.imread('/home/roboworks/Pictures/imgmap_rooms.png')
img_map_rooms[origin_map_img[1]+ round(pose[1]/pix_per_m), origin_map_img[0]+ round(pose[0]/pix_per_m)]

In [ ]:
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]
humanpose=detect_human_to_tf()  #make sure service is running
if humanpose== False:
    
    print ('no human ')
    
else : 
    human_pose,_=tf_man.getTF('human')

    pose=human_pose[:2]
    
    if img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]!=0:#### Yes axes seem to be "flipped" !=0:
            print ('reject point, most likely part of the audience, outside of the arena map')
            
    dists=(pose-np.asarray(xys))
    human_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'human in {human_room}')
    
    robot_pose=get_robot_px()
    dists=(robot_pose-np.asarray(xys))
    robot_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'Robot  in {robot_room}')
    
    if robot_room != human_room: print('maybe false positive... ignoring... ')
    

In [ ]:
img_map_rooms=cv2.imread('/home/roboworks/Pictures/imgmap.png')
img_map_rooms[origin_map_img[1]+ round(pose[1]/pix_per_m), origin_map_img[0]+ round(pose[0]/pix_per_m)]

In [ ]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
#cv2.imwrite('/home/roboworks/Pictures/imgmap.png', img_map)
#img_map_rooms=cv2.imread('/home/roboworks/Pictures/imgmap.png')

cropping = False

x_start, y_start, x_end, y_end = 0, 0, 0, 0

image = img_map_rooms
oriImage = image.copy()
ii = image.copy()
room_no=0
regions=[]
global room_no , regions
def mouse_crop(event, x, y, flags, param):
    # grab references to the global variables
    global x_start, y_start, x_end, y_end, cropping,room_no
    
    # if the left mouse button was DOWN, start RECORDING
    # (x, y) coordinates and indicate that cropping is being
    if event == cv2.EVENT_LBUTTONDOWN:
        

        x_start, y_start, x_end, y_end = x, y, x, y
        cropping = True
        

    # Mouse is Moving
    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping :
            x_end, y_end = x, y

    # if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates
        x_end, y_end = x, y
        cropping = False # cropping is finished

        refPoint = [(x_start, y_start), (x_end, y_end)]

        if len(refPoint) == 2: #when two points were found
            room_no+=1
            print (room_no)
            cv2.rectangle(ii, (x_start, y_start), (x_end, y_end), (255*np.random.rand(),255*np.random.rand(),255*np.random.rand()), -1)
            regions.append(((x_start, y_start), (x_end, y_end)))
            print (regions)
            cv2.putText(ii, f'room{room_no}', (x_end,y_end),cv2.FONT_HERSHEY_SIMPLEX, 1 , (255, 0, 0),cv2.LINE_4)
            cv2.imshow("rooms", ii)

cv2.namedWindow("image")
cv2.setMouseCallback("image", mouse_crop)

while True:

    i = image.copy()

    if not cropping:
        cv2.imshow("image", image)

    elif cropping:
        cv2.rectangle(i, (x_start, y_start), (x_end, y_end), (255, 255,255), 6)
        cv2.imshow("image", i)
    
    
    
    keystroke = cv2.waitKey(0)
    if 32 <= keystroke and keystroke < 128:
        key = chr(keystroke).lower()
        print (key)
        if key=='q':
            cv2.destroyAllWindows()
            
            break


living_room_px_region=np.asarray(regions[0])
kitchen_px_region=np.asarray(regions[1])
bedroom_px_region=np.asarray(regions[2])
dining_room_px_region=np.asarray(regions[3])
    
# close all open windows


In [ ]:
living_room_px_region


In [ ]:
l=np.asarray(regions[0])
living_room_px_region=np.asarray([(241, 222), (322, 328)])
print(l , living_room_px_region)

In [ ]:
iimg=draw_at(px_pose[1],px_pose[0],img_map)
plt.imshow (iimg)

In [ ]:
np.asarray((origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)))

In [ ]:
pose=get_robot_px()
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]

px_pose=np.asarray((origin_map_img[0]+pose[0],origin_map_img[1]+pose[1]))
px_pose

In [ ]:
humanpose=detect_human_to_tf()  #make sure service is running


In [ ]:
##REAL LAB
#living_room_px_region=np.asarray([(422, 190), (494, 254)])
#dining_room_px_region=np.asarray([(419, 144), (497, 196)])
#kitchen_px_region=np.asarray([(385, 185), (425, 243)])
#bedroom_px_region=np.asarray([(320, 173), (372, 238)])


##RC SIM
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]

living_room_px_region=np.asarray([(241, 222), (322, 328)])
kitchen_px_region=np.asarray([(331, 220), (414, 328)])

bedroom_px_region=np.asarray([(331, 164), (412, 208)])
dining_room_px_region=np.asarray([(239, 163), (320, 214)])


def check_room_px(px_pose):
    
    for i in range(4):
        if i==0:
            px_region=living_room_px_region
            region='living_room'
        if i==1:
            px_region=kitchen_px_region
            region='kitchen'
        if i==2:
            px_region=bedroom_px_region
            region='bedroom'
        if i==3:
            px_region=dining_room_px_region
            region='dining_room'
        #print (region,px_region,px_pose)
        if (px_pose[1]< px_region[1,1]) and (px_pose[1]> px_region[0,1]) and (px_pose[0]> px_region[0,0]) and (px_pose[0]< px_region[1,0]) : 
            print (f'in  {region}')
            return region

In [ ]:
humanpose=detect_human_to_tf()  #make sure service is running

if humanpose== False:
    
    print ('no human ')
    
else : 
    human_pose,_=tf_man.getTF('human')

    pose=human_pose[:2]
    px_pose_human=np.asarray(([origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]))
    if img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]!=0:#### Yes axes seem to be "flipped" !=0:
            print ('reject point, most likely part of the audience, outside of the arena map')
    
    pose=get_robot_px()
    px_pose_robot=np.asarray((origin_map_img[0]+pose[0],origin_map_img[1]+pose[1]))
    origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]
    room_robot,room_human=check_room_px(px_pose_robot),check_room_px(np.flip(px_pose_human))

    print ('px robot',px_pose_robot,px_pose_human)
    print('room_robot,room_human',room_robot,room_human)

In [ ]:
##REAL LAB
living_room_px_region=np.asarray([(422, 190), (494, 254)])
dining_room_px_region=np.asarray([(419, 144), (497, 196)])
kitchen_px_region=np.asarray([(385, 185), (425, 243)])
bedroom_px_region=np.asarray([(320, 173), (372, 238)])


##RC SIM
living_room_px_region=np.asarray([(241, 222), (322, 328)])
kitchen_px_region=np.asarray([(331, 220), (414, 328)])

bedroom_px_region=np.asarray([(331, 164), (412, 208)])
dining_room_px_region=np.asarray([(239, 163), (320, 214)])



In [ ]:
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]
humanpose=detect_human_to_tf()  #make sure service is running
humanpose

In [ ]:
human_pose,_=tf_man.getTF('human')

pose=human_pose[:2]
if img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]!=0:#### Yes axes seem to be "flipped" !=0:
        print ('reject point, most likely part of the audience, outside of the arena map')

In [ ]:

if humanpose== False:
    
    print ('no human ')
    
else : 
    human_pose,_=tf_man.getTF('human')

    pose=human_pose[:2]
    
    if img_map[origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m)]!=0:#### Yes axes seem to be "flipped" !=0:
            print ('reject point, most likely part of the audience, outside of the arena map')
            
    dists=(pose-np.asarray(xys))
    human_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'human in {human_room}')
    
    robot_pose=get_robot_px()
    dists=(robot_pose-np.asarray(xys))
    robot_room=room_names[np.linalg.norm(dists, axis=1).argmin()]
    print(f'Robot  in {robot_room}')
    
    if robot_room != human_room: print('maybe false positive... ignoring... ')
    

In [ ]:
robot_pose=get_robot_px()
robot_pose_px=[origin_map_img[1]+ round(robot_pose[1]/pix_per_m),origin_map_img[0]+ round(robot_pose[0]/pix_per_m)]

In [ ]:
pose=get_robot_px()
origin_map_img=[round(img_map.shape[0]*0.5) ,round(img_map.shape[1]*0.5)]

In [ ]:
origin_map_img

In [ ]:
get_robot_px()

In [ ]:
px_pose

In [ ]:
origin_map

In [ ]:
iimg=draw_at(px_pose_robot[0] ,px_pose_robot[1], img_map )
#iimg= draw_at(origin_map_img[1]+ round(pose[1]/pix_per_m),origin_map_img[0]+ round(pose[0]/pix_per_m), img_map)


In [ ]:
plt.imshow(iimg)

In [ ]:
px_pose_human

In [ ]:
next_room= 'living_room'
kl_pose_bedroom=(df[df['child_id_frame']=='bedroom'][['x','y','th']].values.ravel())
kl_room=(df[df['child_id_frame']==next_room][['x','y','th']].values.ravel())
kl_room

In [ ]:
kl_room=(df[df['child_id_frame']==next_room][['x','y','th']].values.ravel())
omni_base.move_base(goal_x=kl_room[0],goal_y=kl_room[1],goal_theta=kl_room[2] )

In [ ]:
trans, _=tf_man.getTF('base_link')
robot_pose=np.asarray(trans[:2])
    
robot_pose

In [ ]:

xys, room_names

In [ ]:
dists=(robot_pose-np.asarray(xys))
robot_room=room_names[np.linalg.norm(dists, axis=0).argmin()]

In [ ]:
robot_room

In [ ]:
xys

In [ ]:
import cv2
import numpy as np
drawing = False
ix,iy = -1,-1

# define mouse callback function to draw circle
def draw_rectangle(event, x, y, flags, param):
   global ix, iy, drawing, img
   if event == cv2.EVENT_LBUTTONDOWN:
      drawing = True
      ix = x
      iy = y
   elif event == cv2.EVENT_LBUTTONUP:
      drawing = False
      cv2.rectangle(img, (ix, iy),(x, y),(0, 255, 255),-1)

# Create a black image
img = np.zeros((512,700,3), np.uint8)

# Create a window and bind the function to window
cv2.namedWindow("Rectangle Window")

# Connect the mouse button to our callback function
cv2.setMouseCallback("Rectangle Window", draw_rectangle)


In [ ]:
# define mouse callback function to draw circle
def draw_rectangle(event, x, y, flags, param):
   global ix, iy, drawing, img
   if event == cv2.EVENT_LBUTTONDOWN:
      drawing = True
      ix = x
      iy = y
   elif event == cv2.EVENT_LBUTTONUP:
      drawing = False
      cv2.rectangle(img, (ix, iy),(x, y),(0, 255, 255),-1)

# Create a black image
img = np.zeros((512,700,3), np.uint8)

# Create a window and bind the function to window
cv2.namedWindow("Rectangle Window")

# Connect the mouse button to our callback function
cv2.setMouseCallback("Rectangle Window", draw_rectangle)

# display the window
while True:
   cv2.imshow("Rectangle Window", img)
   if cv2.waitKey(10) == 27:
        break
      

In [ ]:
import numpy as np
import cv2

In [ ]:
from interactive_markers.interactive_marker_server import *
from visualization_msgs.msg import *

In [ ]:
int_marker = InteractiveMarker()

In [ ]:
InteractiveMarkerControl

In [ ]:
control = InteractiveMarkerControl()

In [ ]:
control

In [ ]:

import tf
